In [ ]:
import os
import datetime as di
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
for dirname, _ , filenames in os.walk('/content/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/songs.csv
/content/song_extra_info.csv
/content/sample_submission.csv
/content/members.csv
/content/train.csv
/content/test.csv
/content/.config/active_config
/content/.config/.last_update_check.json
/content/.config/.last_survey_prompt.yaml
/content/.config/.last_opt_in_prompt.yaml
/content/.config/default_configs.db
/content/.config/gce
/content/.config/config_sentinel
/content/.config/logs/2023.11.21/14.24.29.745469.log
/content/.config/logs/2023.11.21/14.18.34.556141.log
/content/.config/logs/2023.11.21/14.24.39.684965.log
/content/.config/logs/2023.11.21/14.21.40.489438.log
/content/.config/logs/2023.11.21/14.24.40.484653.log
/content/.config/logs/2023.11.21/14.21.30.762319.log
/content/.config/configurations/config_default
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_train_small.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/mnist_test.csv


In [ ]:
train_df= pd.read_csv('./train.csv')
test_df=pd.read_csv('./test.csv')
song_df=pd.read_csv('./songs.csv')
members=pd.read_csv('./members.csv')
print(train_df.shape,test_df.shape,song_df.shape,members.shape)

(7768, 6) (15573, 6) (21099, 7) (34403, 7)


In [ ]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1.0


In [ ]:
train_df.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target'],
      dtype='object')

In [ ]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7768 entries, 0 to 7767
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   msno                7768 non-null   object 
 1   song_id             7767 non-null   object 
 2   source_system_tab   7753 non-null   object 
 3   source_screen_name  7577 non-null   object 
 4   source_type         7753 non-null   object 
 5   target              7767 non-null   float64
dtypes: float64(1), object(5)
memory usage: 364.2+ KB
None


In [ ]:
train_df.describe()

,target
count,7767.000000
mean,0.787434
std,0.409149
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
test_df.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [ ]:
test_df.columns

Index(['id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type'],
      dtype='object')

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15573 entries, 0 to 15572
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  15573 non-null  int64 
 1   msno                15573 non-null  object
 2   song_id             15572 non-null  object
 3   source_system_tab   15535 non-null  object
 4   source_screen_name  14704 non-null  object
 5   source_type         15486 non-null  object
dtypes: int64(1), object(5)
memory usage: 730.1+ KB


In [ ]:
test_df.describe()

,id
count,15573.000000
mean,7786.000000
std,4495.682206
min,0.000000
25%,3893.000000
50%,7786.000000
75%,11679.000000
max,15572.000000


In [ ]:
song_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [ ]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21099 entries, 0 to 21098
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   song_id      21099 non-null  object 
 1   song_length  21099 non-null  int64  
 2   genre_ids    20737 non-null  object 
 3   artist_name  21099 non-null  object 
 4   composer     12291 non-null  object 
 5   lyricist     6006 non-null   object 
 6   language     21098 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.1+ MB


In [ ]:
train_df=pd.merge(train_df,song_df,on='song_id',how='left')
df_test=pd.merge(test_df,song_df,on='song_id',how='left')

train_df=pd.merge(train_df,members,on='msno',how='left')
df_test=pd.merge(test_df,members,on='msno',how='left')

train_df

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,7.0,20120102.0,20171005.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,24.0,female,9.0,20110525.0,20170911.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,24.0,female,9.0,20110525.0,20170911.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,24.0,female,9.0,20110525.0,20170911.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,7.0,20120102.0,20171005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7763,7H8Teyq/bk0vHVwbjT2+Otaz8vHTgdJHmfxwYm2P/K8=,FynUyq0+drmIARmK1JZ/qcjNZ7DKkqTY6/0O0lTzNUI=,my library,Local playlist more,local-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,4.0,20160617.0,20171001.0
7764,7H8Teyq/bk0vHVwbjT2+Otaz8vHTgdJHmfxwYm2P/K8=,TLU46LbiJ+1mmJr4Le4Z5eaUQq3TiXgdZpE5EtR4WZI=,my library,Local playlist more,local-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,4.0,20160617.0,20171001.0
7765,7H8Teyq/bk0vHVwbjT2+Otaz8vHTgdJHmfxwYm2P/K8=,reXuGcEWDDCnL0K3Th//3DFG4S1ACSpJMzA+CFipo1g=,my library,Local playlist more,local-playlist,1.0,219324.0,458,周湯豪 (NICKTHEREAL),周湯豪,周湯豪\崔惟楷,3.0,1.0,0.0,NaN,4.0,20160617.0,20171001.0
7766,7H8Teyq/bk0vHVwbjT2+Otaz8vHTgdJHmfxwYm2P/K8=,Z+mhovcYbExPp66GnBecKttDRM5S5YCAw9vzJACsQAU=,my library,Local playlist more,local-playlist,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,4.0,20160617.0,20171001.0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7768 entries, 0 to 7767
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   msno                    7768 non-null   object 
 1   song_id                 7767 non-null   object 
 2   source_system_tab       7753 non-null   object 
 3   source_screen_name      7577 non-null   object 
 4   source_type             7753 non-null   object 
 5   target                  7767 non-null   float64
 6   song_length             618 non-null    float64
 7   genre_ids               601 non-null    object 
 8   artist_name             618 non-null    object 
 9   composer                479 non-null    object 
 10  lyricist                386 non-null    object 
 11  language                618 non-null    float64
 12  city                    7767 non-null   float64
 13  bd                      7767 non-null   float64
 14  gender                  4572 non-null   

In [ ]:
train_df.describe()

,target,song_length,language,city,bd,registered_via,registration_init_time,expiration_date
count,7767.000000,618.000000,618.000000,7767.000000,7767.000000,7767.000000,7.767000e+03,7.767000e+03
mean,0.787434,245672.668285,16.202265,7.610145,16.797605,6.901893,2.012598e+07,2.017206e+07
std,0.409149,46077.572016,20.304957,6.525624,15.554188,2.180948,2.813990e+04,3.787005e+03
min,0.000000,66247.000000,-1.000000,1.000000,0.000000,3.000000,2.004033e+07,2.016023e+07
25%,1.000000,219324.000000,3.000000,1.000000,0.000000,7.000000,2.011053e+07,2.017091e+07
50%,1.000000,239400.500000,3.000000,5.000000,20.000000,7.000000,2.013072e+07,2.017093e+07
75%,1.000000,273170.750000,31.000000,13.000000,29.000000,9.000000,2.015050e+07,2.017101e+07
max,1.000000,491728.000000,52.000000,22.000000,111.000000,13.000000,2.016113e+07,2.019022e+07


In [ ]:
train_df.isnull().sum()

msno                         0
song_id                      1
source_system_tab           15
source_screen_name         191
source_type                 15
target                       1
song_length               7150
genre_ids                 7167
artist_name               7150
composer                  7289
lyricist                  7382
language                  7150
city                         1
bd                           1
gender                    3196
registered_via               1
registration_init_time       1
expiration_date              1
dtype: int64

In [ ]:
dtypes = pd.DataFrame(train_df.dtypes,columns=["Data Type"])

dtypes["Unique Values"]=train_df.nunique().sort_values(ascending=True)

dtypes["Null Values"]=train_df.isnull().sum()

dtypes["% null Values"]=train_df.isnull().sum()/len(train_df)

dtypes.sort_values(by="Null Values" , ascending=False).style.background_gradient(cmap='YlOrRd',axis=0)

,Data Type,Unique Values,Null Values,% null Values
lyricist,object,171,7382,0.950309
composer,object,233,7289,0.938337
genre_ids,object,32,7167,0.922631
song_length,float64,369,7150,0.920443
artist_name,object,271,7150,0.920443
language,float64,7,7150,0.920443
gender,object,2,3196,0.411432
source_screen_name,object,17,191,0.024588
source_system_tab,object,7,15,0.001931
source_type,object,10,15,0.001931


In [ ]:
for col in train_df.select_dtypes(include=['object']).columns:
    train_df[col]=train_df[col].fillna(value='unknown')

for col in train_df.select_dtypes(exclude=['object']).columns:
    train_df[col]=train_df[col].fillna(value= train_df[col].mean())

train_df.isnull().sum()

msno                      0
song_id                   0
source_system_tab         0
source_screen_name        0
source_type               0
target                    0
song_length               0
genre_ids                 0
artist_name               0
composer                  0
lyricist                  0
language                  0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
expiration_date           0
dtype: int64

In [ ]:
train_df['registration_init_time'] = pd.to_datetime(train_df['registration_init_time'], format='%Y%m%d', errors='coerce')

train_df['registration_init_time_year']=train_df['registration_init_time'].dt.year
train_df['registration_init_time_month']=train_df['registration_init_time'].dt.month
train_df['registration_init_time_day']=train_df['registration_init_time'].dt.day


train_df['expiration_date'] = pd.to_datetime(train_df['expiration_date'],  format='%Y%m%d', errors='coerce')

train_df['expiration_date_year']=train_df['expiration_date'].dt.year
train_df['expiration_date_month']=train_df['expiration_date'].dt.month
train_df['expiration_date_day']=train_df['expiration_date'].dt.day

In [ ]:
# Dates to categoty
train_df['registration_init_time'] = train_df['registration_init_time'].astype('category')
train_df['expiration_date'] = train_df['expiration_date'].astype('category')

In [ ]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,...,gender,registered_via,registration_init_time,expiration_date,registration_init_time_year,registration_init_time_month,registration_init_time_day,expiration_date_year,expiration_date_month,expiration_date_day
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,245672.668285,unknown,unknown,unknown,...,unknown,7.0,2012-01-02,2017-10-05,2012.0,1.0,2.0,2017.0,10.0,5.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1.0,245672.668285,unknown,unknown,unknown,...,female,9.0,2011-05-25,2017-09-11,2011.0,5.0,25.0,2017.0,9.0,11.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1.0,245672.668285,unknown,unknown,unknown,...,female,9.0,2011-05-25,2017-09-11,2011.0,5.0,25.0,2017.0,9.0,11.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1.0,245672.668285,unknown,unknown,unknown,...,female,9.0,2011-05-25,2017-09-11,2011.0,5.0,25.0,2017.0,9.0,11.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1.0,245672.668285,unknown,unknown,unknown,...,unknown,7.0,2012-01-02,2017-10-05,2012.0,1.0,2.0,2017.0,10.0,5.0


In [ ]:
for col in train_df.select_dtypes(include=['object']).columns:
    train_df[col]=train_df[col].astype('category')

for col in train_df.select_dtypes(include=['category']).columns:
    train_df[col]=train_df[col].cat.codes
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7768 entries, 0 to 7767
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   msno                          7768 non-null   int16  
 1   song_id                       7768 non-null   int16  
 2   source_system_tab             7768 non-null   int8   
 3   source_screen_name            7768 non-null   int8   
 4   source_type                   7768 non-null   int8   
 5   target                        7768 non-null   float64
 6   song_length                   7768 non-null   float64
 7   genre_ids                     7768 non-null   int8   
 8   artist_name                   7768 non-null   int16  
 9   composer                      7768 non-null   int16  
 10  lyricist                      7768 non-null   int16  
 11  language                      7768 non-null   float64
 12  city                          7768 non-null   float64
 13  bd 

In [ ]:
train_df=train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7767 entries, 0 to 7766
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   msno                          7767 non-null   int16  
 1   song_id                       7767 non-null   int16  
 2   source_system_tab             7767 non-null   int8   
 3   source_screen_name            7767 non-null   int8   
 4   source_type                   7767 non-null   int8   
 5   target                        7767 non-null   float64
 6   song_length                   7767 non-null   float64
 7   genre_ids                     7767 non-null   int8   
 8   artist_name                   7767 non-null   int16  
 9   composer                      7767 non-null   int16  
 10  lyricist                      7767 non-null   int16  
 11  language                      7767 non-null   float64
 12  city                          7767 non-null   float64
 13  bd 

In [ ]:
X=train_df.drop(columns=['target'])
Y=train_df['target']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
y_test

1421    1.0
2746    1.0
3473    1.0
6688    0.0
3000    1.0
       ... 
4745    1.0
5537    0.0
904     1.0
1446    1.0
7284    1.0
Name: target, Length: 1554, dtype: float64

In [ ]:
RF = RandomForestClassifier(n_jobs=1)
RF.fit(x_train, y_train)

RandomForestClassifier(n_jobs=1)

In [ ]:
RF_Score = RF.score(x_test,y_test)
print(f'Accuracy of Random Forest Classifier : {RF_Score*100:.2f}%')

Accuracy of Random Forest Classifier : 85.33%
